In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [4]:
# 1. Load and split the data
iris = load_iris()
X = iris.data           # shape (150, 4)
y = iris.target         # shape (150,)

# 90% train, 10% test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.9, random_state=42, stratify=y
)

# Convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_test  = torch.tensor(y_test,  dtype=torch.long)

# Create DataLoader for batching
batch_size = 16
train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# 2. Define the model
class IrisNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1  = nn.Linear(4, 128)  # input layer -> hidden layer
        self.relu = nn.ReLU()
        self.fc2  = nn.Linear(128, 3)  # hidden layer -> output layer (3 classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = IrisNet()

# 3. Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()      # for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop
epochs = 10
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_X.size(0)

    epoch_loss = running_loss / len(train_ds)
    print(f"Epoch {epoch:2d}/{epochs}, Loss: {epoch_loss:.4f}")

    # 5. Evaluation on test set
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, preds = torch.max(outputs, dim=1)
        accuracy = (preds == y_test).float().mean()
        print(f"Test Accuracy: {accuracy * 100:.2f}%\n")

Epoch  1/10, Loss: 1.1236
Test Accuracy: 86.67%

Epoch  2/10, Loss: 0.9175
Test Accuracy: 73.33%

Epoch  3/10, Loss: 0.7974
Test Accuracy: 66.67%

Epoch  4/10, Loss: 0.6967
Test Accuracy: 86.67%

Epoch  5/10, Loss: 0.6243
Test Accuracy: 93.33%

Epoch  6/10, Loss: 0.5686
Test Accuracy: 86.67%

Epoch  7/10, Loss: 0.5260
Test Accuracy: 100.00%

Epoch  8/10, Loss: 0.4861
Test Accuracy: 93.33%

Epoch  9/10, Loss: 0.4571
Test Accuracy: 100.00%

Epoch 10/10, Loss: 0.4307
Test Accuracy: 93.33%

